<font size=5 color='cornflowerblue'>***Animations* of planetary systems!**</font>

In the previous interactive workbook, we explored the orbits of planets around their stars using the interactive widgets. In this notebook, we will explore some orbits using animations and we will create our own gifs.

In [1]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
import imageio # this is new! We're going to be saving our animations as gifs!

# so we can make plots
%matplotlib inline 
%run tools.ipynb  # Some behind-the-scenes tools to simplify some complicated code


We created a definition in the previous notebook to start a new simulation and add a star! We have to redefine that function in this notebook now:


In [2]:
def start_new_sim(star_mass): # We have to give our def a unique name, and we can pass parameters to it
    sim = rebound.Simulation()
    sim.G = 4.*pi**2.
    sim.units = ('yr', 'AU', 'Msun')
    sim.add(m=star_mass)  # Here, we use the variables we passed to the definition to initialize our star
    return sim # We always need to return from our definition; here, we want to pass back our simulation


We'll start with the plot that I asked you to make last time! 

Rebound has the capability to search the internet for the orbital parameters of some well known objects, like the planets. It does take a little longer though! Rebound gets the data from [this website](https://ssd.jpl.nasa.gov/horizons.cgi#top). You can try finding objects on this page yourself, if you are interested. Ask me if you need help! 

In [3]:
t_final = 5 # Length of simulation in years
N_frames = 200 # Number of frames in the animation

# Start new simulation - you only get 1 widget per simulation, so if you want a new widget, you need a new simulation
sim = start_new_sim(1.) # We call the definition, and we must pass the required elements
##
# use the documentation to figure out how to add planets by having Rebound look them up
##
sim.move_to_com()


# And now integrate!
integrate_with_widget(sim, t_final, N_frames, 22) # the 22 stands for the scale used. Increase this number if you want to zoom out!


Widget(N=1, count=2, height=500.0, orbits=True, orientation=(0.0, 0.0, 0.0, 1.0), overlay='REBOUND (ias15), N=…

/opt/anaconda3/lib/python3.8/site-packages/rebound/simulation.py:716: RuntimeWarning: At least 10 predictor corrector loops in IAS15 did not converge. This is typically an indication of the timestep being too large.
  warnings.warn(msg[1:], RuntimeWarning)


We can also just add these planets manually! In the tools.ipynb notebook, I created functions that would add each of the planets to a simulation! You can use these functions by typing: `addEarth()` or `addJupiter()` etc. There are additional functions, `addInnerPlanets()` to add Mercury, Venus, Earth, and Mars, and `addOuterPlanets()` for Jupiter, Saturn, Uranus, and Neptune. Pluto can be added with `addPluto()`. 

In [4]:
t_final = 50 # Length of simulation in years
N_frames = 200 # Number of frames in the animation

# Start new simulation - you only get 1 widget per simulation, so if you want a new widget, you need a new simulation
sim = start_new_sim(1.) # We call the definition, and we must pass the required elements
addOuterPlanets(sim) # this function adds Jupiter, Saturn, Uranus, and Neptune!
addPluto(sim)
sim.move_to_com()


# And now integrate!
integrate_with_widget(sim, t_final, N_frames, 22) # the 22 stands for the scale used. Increase this number if you want to zoom out!


Widget(N=6, count=2, height=500.0, orbit_data=b"M\x8d\x0e\xbb\xecu\xf9;\xc7\xe4\xac\xb7\xcc\x7f\xa6@['H=^z\x89…

This is a good example of an inclined orbit! Pluto got knocked around a lot by the large outer planets when it was a young dwarf planet, so it's orbit is very different than those of the planets. 

<font color='cornflowerblue'>**Can you tell if Pluto could ever accidentally run into Neptune?**</font>
<br><br>
<br><br>
<br><br>
<br><br>

Let's create a plot that shows all the inner and outer planets, Pluto, *and* Asbolus, which is a large asteroid that has an *unstable orbit* close to the outer planets! Move the image around until you have a good understanding of where each planet is, and how the orbits of pluto and asbolus compare to the orbits of the planets!

In [ ]:
# This plot will include all the inner planets and the 

t_final = 5 # Length of simulation in years
N_frames = 200 # Number of frames in the animation

# Start new simulation - you only get 1 widget per simulation, so if you want a new widget, you need a new simulation
sim = start_new_sim(1.) # We call the definition, and we must pass the required elements
addInnerPlanets(sim)
# addEarth(sim) # a function I made to add earth!
addOuterPlanets(sim) # this function adds Jupiter, Saturn, Uranus, and Neptune!
addPluto(sim)
addAsbolus(sim)
sim.move_to_com()


# And now integrate!
integrate_with_widget(sim, t_final, N_frames, 22) # the 22 stands for the scale used. Increase this number if you want to zoom out!


<font size=4 color='cornflowerblue'> **How do these simulations work?** </font>

This simulation takes small steps forward in time. At each step, the integrator will use the laws of gravity to estimate how the positions and velocities of **every** body in the simulation change. At the next step, it will build from the positions and velocities of the star and planets calculated in the previous step.

To mimic this behavior ourselves in order to create an animation (or video), the cell below uses a `for` loop to incrementally evolve our simulation forward in time. We achieve this by specifying a time step (`dt`) and the number of steps in our loop (`N_steps`). At each step, we add `dt` to the simulation's current time and "integrate" or evolve to this new time. Remember that `sim.integrate` takes the end time, which we have assigned to the variable `new_time`. We also generate an animation frame at each step, which will then be stitched together into an animation similar to the widget above. 

Note that we name our animation `ani` - the word `animation` belongs to a package we imported, so we can't use it as a variable name.

In [6]:
%%capture 
# This cell produces a lot of unnecessary output, so we'll capture that output instead of printing it out.
# The next code cell will actually display the animation.

dt = 0.1 # Let's go forward a tenth of a year each step
N_steps = 20 # Let's integrate it forward for 20 steps, so 20*0.1 = 2 years

# Start a fresh simulation
sim = start_new_sim(1) # We call the definition, and we must pass the required elements
sim.add(m=3.E-6, a=2., e=0.8, inc=radians(20.), f=radians(45.), hash='Planet')
sim.move_to_com()

# We need to create a figure (fig) and axes (ax) for the animation
fig,ax = plt.subplots(1,1,figsize=(5,5))

# The animation is created from a list of frames
frames = []

for i in range(N_steps):
    
    new_time = sim.t+dt # The time we want to integrate to
    sim.integrate(new_time) # And integrate!
    
    frames.append(make_rebound_frame(ax,sim)) # Add the new frame to the list for animation
       
# Create the animation
ani = animation.ArtistAnimation(fig, frames)


Now let's display our animation! Use the row of control buttons below the animation to adjust the animation speed and move forward/backward through the integration steps.

In [7]:
ani

<font color='cornflowerblue' size=4> **Use an animation to figure out what Jupiter's Orbital Period**</font>

Since the only information that is used in constructing this simulation is info about the orbit, the amount of time it takes for Jupiter to orbit the sun is only related to it's orbital parameters! In particular, the orbital period is closely-related to the distance from the sun. Can you use the animation below to determine Jupiter's orbital period?

In [ ]:
%%capture 
# This cell produces a lot of unnecessary output, so we'll capture that output instead of printing it out.
# The next code cell will actually display the animation.

dt = 0.1 # Let's integrate a tenth of a year each step
N_steps = 200 # Let's integrate it forward for 20 steps, so 20*0.1 = 2 years

# Start a fresh simulation
sim = start_new_sim(1.) 
##
# fill this part in
##
sim.move_to_com()

# We need to create a figure (fig) and axes (ax) for the animation
fig,ax = plt.subplots(1,1,figsize=(5,5))

# The animation is created from a list of frames
frames = []

for i in range(N_steps):
    
    new_time = sim.t+dt # The time we want to integrate to
    sim.integrate(new_time) # And integrate!
    
    frames.append(make_rebound_frame(ax,sim)) # Add the new frame to the list for animation
       
# Create the animation
ani = animation.ArtistAnimation(fig, frames)

In [ ]:
ani

<font color='cornflowerblue' size=4>**Now you!**</font>

Make an animation with 20 frames again, but use the orbits of the outer planets and Pluto. *Be warned: the more planets you add, the longer the integration will take! It will take about 30 seconds for this to run.* 

Take the following steps to complete this: 
1. change the time steps to be one year long, since the outer planets do not move very far in one year.
2. start a new simulation, with the Sun, Jupiter, Saturn, Uranus, Neptune, and Pluto.
3. Run for ~30 seconds, then run the cell that shows the animation


<font color='cornflowerblue' size=5>**Gifs**</font>

Alternatively, we can turn this animation into a gif that we can use to show our friends or family! We want to go through a series of time steps, make a plot for each time step, and save a picture at each step. 

In [ ]:
%%capture
# this for loop will print out the image that we're saving. if you'd like to see them, uncomment the line above

dt = 1 # Let's go forward a full year each step
N_steps = 100 # Let's integrate it forward for 20 steps, so 20 years

sim = start_new_sim(1) # We call the definition, and we must pass the required elements
addOuterPlanets(sim)
addPluto(sim)

# As before, we have to re-center the simulation before evolving it forward in time
sim.move_to_com()

for i in range(N_steps):
    
    new_time = sim.t+dt # The time we want to integrate to
    sim.integrate(new_time) # And integrate!
    
    rebound.OrbitPlot(sim, unitlabel="(AU)", color=True)
    plt.savefig('figs/step_'+str(i)+'.png')
    

If you go back to the tab with your google drive(or if using Binder, the tab that says 'Home'), you'll see a folder listed that reads 'figs'. If you open that folder, you'll see a list of a bunch of figures! If you open one, you'll see an image of an orbit we simulated. Now, we have to stitch all of those images into a gif! 

We can do that by running the cell below, which goes through each image and stitches it to the previous one, and then exports the movie as a gif!

In [ ]:
images = []
filenames = ['figs/step_'+str(i)+'.png' for i in range(N_steps)]
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('movie.gif', images)

You can find this 'movie.gif' file in the other table where you saw the single image. You can download this image by: (Binder:) clicking on the check box, and going to the top and pressing download (Google Docs:) Right click on the file and click download. Let me know if you have trouble with this! 

<font color='cornflowerblue'>**Now, you can make whatever gifs you want!**</font> You can simulate the inner planets, the outer planets, all the planets, include Pluto, include any asteroids or comets you want (remember you can use the automated feature that Rebound has to look up all these orbital parameters easily!)

You could also try: 
- making gifs of a bunch of planets that are all the same, except differ by one orbital parameter
- finding an example of some orbits that result in a planet getting kicked out of it's planetary system
- seeing what happens when you increase or decrease the mass of the planets


<sup> Notebook written by [Katie Chamberlain](katiechambe.github.io) in 2019. </sup>  